#### 輸入起始點、終點座標、法向量，生成掃除路徑

In [29]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir,gp_Circ, gp_Ax2, gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Display.SimpleGui import init_display
import numpy as np

def grinding_area(point_start,point_end,normal_vector):
    #根據起始、終點生成線段
    point_s=gp_Pnt(float(point_start[0]),float(point_start[1]), float(point_start[2]))
    point_e=gp_Pnt(float(point_end[0]),float(point_end[1]), float(point_end[2]))
    aSegment = GC_MakeSegment(point_s, point_e )
    anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
    aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())

    #input
    #normal_vector
    magnitude = np.linalg.norm(normal_vector)# 將法向量除以其範數以得到單位法向量
    normal_vector = normal_vector/ magnitude

    #two point_position
    tan_vector=point_start-point_end

    #direction_1
    dir_1= np.cross(normal_vector, tan_vector)
    magnitude = np.linalg.norm(dir_1)# 將向量除以其範數以得到單位向量
    dir_1 = dir_1/ magnitude

    # 定義截面為方形
    # 定義方形的四個頂點
    depth=0.01#每次研磨深度
    width=10#研磨寬度
    point_1=point_start+normal_vector*depth+dir_1*width
    point_2=point_start+normal_vector*depth-dir_1*width
    point_3=point_start-normal_vector*50-dir_1*width
    point_4=point_start-normal_vector*50+dir_1*width
    # point_1 = [round(coord, 3) for coord in (point_start + normal_vector * depth + dir_1 * width)]
    # point_2 = [round(coord, 3) for coord in (point_start + normal_vector * depth - dir_1 * width)]
    # point_3 = [round(coord, 3) for coord in (point_start - normal_vector * 50 - dir_1 * width)]
    # point_4 = [round(coord, 3) for coord in (point_start - normal_vector * 50 + dir_1 * width)]

    p1 = gp_Pnt(point_1[0], point_1[1], point_1[2])
    p2 = gp_Pnt(point_2[0], point_2[1], point_2[2])
    p3 = gp_Pnt(point_3[0], point_3[1], point_3[2])
    p4 = gp_Pnt(point_4[0], point_4[1], point_4[2])
    point_starts=[p1,p2,p3,p4]

    point_5=point_end+normal_vector*depth+dir_1*width
    point_6=point_end+normal_vector*depth-dir_1*width
    point_7=point_end-normal_vector*50-dir_1*width
    point_8=point_end-normal_vector*50+dir_1*width
    # point_5 = [round(coord, 3) for coord in (point_end + normal_vector * depth + dir_1 * width)]
    # point_6 = [round(coord, 3) for coord in (point_end + normal_vector * depth - dir_1 * width)]
    # point_7 = [round(coord, 3) for coord in (point_end - normal_vector * 50 - dir_1 * width)]
    # point_8 = [round(coord, 3) for coord in (point_end - normal_vector * 50 + dir_1 * width)]
    p5 = gp_Pnt(point_5[0], point_5[1], point_5[2])
    p6 = gp_Pnt(point_6[0], point_6[1], point_6[2])
    p7 = gp_Pnt(point_7[0], point_7[1], point_7[2])
    p8 = gp_Pnt(point_8[0], point_8[1], point_8[2])
    # p7 = gp_Pnt(point_7[0], point_7[1], point_7[2])
    # p8 = gp_Pnt(point_8[0], point_8[1], point_8[2])
    point_ends=[p5,p6,p7,p8]

    # 使用這些頂點創建方形的邊緣
    edge1 = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
    edge2 = BRepBuilderAPI_MakeEdge(p2, p3).Edge()
    edge3 = BRepBuilderAPI_MakeEdge(p3, p4).Edge()
    edge4 = BRepBuilderAPI_MakeEdge(p4, p1).Edge()

    # 使用這些邊緣創建方形的線框
    square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3, edge4).Wire()


    # 將線框轉換為面，作為截面

    F = BRepBuilderAPI_MakeFace(square_wire, True)

    S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟

    return S.Shape(),point_starts,point_ends

#### 等比例縮放

In [30]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.gp import gp_Trsf,gp_Pnt,gp_Vec
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Display.OCCViewer import rgb_color
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

def zoom(model,m_center,mag):
    print(model,m_center,mag)
    t1 = gp_Trsf()
    #物件質心位置,單位mm
    t1.SetScale(m_center, mag)

    # 对形状进行缩放
    scaled_box = BRepBuilderAPI_Transform(model, t1, True).Shape()


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # display.DisplayShape(scaled_box, update=True, color=rgb_color(0, 0, 0))
    # display.DisplayShape(model, update=True, color=rgb_color(1, 0, 0))
    # start_display()
    return scaled_box


#### 掃除

In [31]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display
from OCC.Core.IGESControl import IGESControl_Reader

def grind(model,S):
    # 创建一个几何切割对象
    cut_algo = BRepAlgoAPI_Cut(model,S)
    # 执行几何切割操作
    cut_result = cut_algo.Shape()
    return cut_result


In [32]:
def grind(model, S):
    print(f"Model type: {type(model)}")
    print(f"Cut shape type: {type(S)}")
    
    cut_algo = BRepAlgoAPI_Cut(model, S)
    cut_algo.Build()
    cut_result = cut_algo.Shape()
    
    # if cut_result.IsNull():
    #     print("Warning: Cut result is None.")
    #     return None
    
    return cut_result

#### 計算表面積

In [33]:
from OCC.Core.BRepGProp import brepgprop_SurfaceProperties
from OCC.Core.GProp import GProp_GProps
from OCC.Core.TopoDS import TopoDS_Shape

def calculate_surface_area(shape):
    # 將 shape 轉換為 TopoDS_Shape
    topods_shape = TopoDS_Shape(shape)

    # 初始化表面屬性
    surface_props = GProp_GProps()

    # 計算表面屬性
    brepgprop_SurfaceProperties(topods_shape, surface_props)
    

    # 獲取表面積
    surface_area = surface_props.Mass()

    return surface_area

#### 讀取路徑資料

In [34]:
points = []  # 创建一个空列表来存储解析后的reward值
with open(r"C:\alan\ITRI\ITRI-test\trajectory\complex_3_20240907.txt", "r") as file:
    for line in file:
        # 输出每一行内容，以确保正确读取
        print("Line:", line)
        try:
            # 从每一行中解析出8个值
            face, trajc, x, y, z, rx, ry, rz = line.split()
            # 将解析后的值作为一个元组添加到rewards列表中
            points.append((face, trajc, x, y, z, rx, ry, rz))
        except ValueError:
            # 如果发生拆包错误，输出错误信息
            print("Error: Unable to unpack values from line:", line)

# 输出解析后的rewards列表
print("points =", points)

Line: 1	1	51.86	-0.00	-22.41	0.81	-0.00	-0.59

Line: 1	1	54.81	-11.30	-15.74	0.87	-0.18	-0.45

Line: 1	1	56.88	-15.10	-8.35	0.93	-0.25	-0.26

Line: 1	1	58.31	-14.08	-0.78	0.97	-0.23	-0.03

Line: 1	1	58.42	-13.68	-0.00	0.97	-0.23	0.00

Line: 1	2	38.80	-0.00	-35.58	0.61	-0.00	-0.79

Line: 1	2	41.33	-12.69	-31.90	0.64	-0.20	-0.74

Line: 1	2	44.00	-22.45	-25.54	0.68	-0.35	-0.64

Line: 1	2	46.28	-28.53	-18.63	0.73	-0.45	-0.51

Line: 1	2	48.29	-31.96	-11.29	0.78	-0.52	-0.34

Line: 1	2	49.95	-32.79	-3.84	0.83	-0.54	-0.13

Line: 1	2	50.66	-32.14	-0.00	0.84	-0.54	0.00

Line: 1	3	26.95	-0.00	-43.01	0.45	-0.00	-0.89

Line: 1	3	28.49	-8.85	-41.48	0.47	-0.14	-0.87

Line: 1	3	30.99	-20.11	-36.96	0.49	-0.32	-0.81

Line: 1	3	33.35	-28.61	-31.25	0.52	-0.44	-0.73

Line: 1	3	35.58	-35.08	-24.85	0.55	-0.54	-0.63

Line: 1	3	37.68	-39.77	-17.91	0.60	-0.63	-0.50

Line: 1	3	37.80	-40.00	-17.48	0.60	-0.63	-0.49

Line: 1	4	15.70	-0.00	-47.53	0.29	-0.00	-0.96

Line: 1	4	16.68	-6.10	-46.87	0.30	-0.11	-0.95

Line:

In [35]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir,gp_Circ, gp_Ax2, gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Display.SimpleGui import init_display
import numpy as np
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox

def grinding_corner_area(point_starts,point_ends):
    #根據起始、終點生成線段

    p1=point_starts[0]
    p2=point_starts[1]
    p3=point_starts[2]
    p4=point_starts[3]
    p5=point_ends[0]
    p6=point_ends[1]
    p7=point_ends[2]
    p8=point_ends[3]

   
    if p3.X()!=p7.X() or p3.Y()!=p7.Y() or p3.Z()!=p7.Z():
        aSegment = GC_MakeSegment(p1,p2)
        anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
        aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())
        edge1 = BRepBuilderAPI_MakeEdge(p1, p4).Edge()
        edge2 = BRepBuilderAPI_MakeEdge(p4, p8).Edge()
        edge3 = BRepBuilderAPI_MakeEdge(p8, p1).Edge()

        # 使用這些邊緣創建方形的線框
        square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3).Wire()

        # 將線框轉換為面，作為截面
        F = BRepBuilderAPI_MakeFace(square_wire, True)
        S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟
        print("ok")

    else: 
        S =  BRepPrimAPI_MakeBox(0.00001, 0.00001, 0.00001)



    return S.Shape()

#### 組合掃除路徑

In [36]:
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
max_value_a = max(int(point[0]) for point in points)
# print("Maximum value of point[1]:", max_value_a)
max_value_b = max(int(point[1]) for point in points)
# path_result = None 
# path_result_2=None
surface_result=None
for a in range(1, max_value_a+1): 
    same_surface_points = [point for point in points if float(point[0]) ==  a]#提取同一面上的點
    # same_surface_points=[point for point in points if point[0] == '2']#提取同一面上的點
    for b in range(1, max_value_b+1): 
        same_path_points = [point for point in same_surface_points if float(point[1]) ==  b]#提取同一路徑上的點
        print(same_path_points)
        if len(same_path_points)==0:
            continue

        #same_path_points = [point for point in same_surface_points if point[1] == '1']#提取同一路徑上的點
        for i, point in enumerate(same_path_points):
            point = [float(item) for item in point]  # 将字符串转换为浮点数
            # print("Current Point:", point)


            
            point_end=np.array((point[2],point[3],point[4]))
            # print(point_end)
            normal_vector_end=np.array((point[5],point[6],point[7]))
            # print(normal_vector_end)
            if i > 0:
                previous_point = [float(item) for item in same_path_points[i - 1]]  # 获取前一个 point 的信息
                # print("Previous Point:", previous_point)
                point_start=np.array((previous_point[2],previous_point[3],previous_point[4]))
                normal_vector_start=np.array((previous_point[5],previous_point[6],previous_point[7]))
                # print(normal_vector_start)
                normal_vector=(normal_vector_end+normal_vector_start)/2

                are_equal = np.array_equal(point_start, point_end)
                # if are_equal==1:
                #     point_start[0]=point_start[0]+0.001
                if are_equal==0:
                    # print(point_start,point_end,normal_vector)
                    S,point_starts,point_ends=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
                    p1=point_starts[0]
                    p2=point_starts[1]
                    p3=point_starts[2]
                    p4=point_starts[3]
                    p5=point_ends[0]
                    p6=point_ends[1]
                    p7=point_ends[2]
                    p8=point_ends[3]
                    # print("-------")
                    # print("pointstart")
                    # print(p1.X(),p1.Y(),p1.Z())
                    # print(p2.X(),p2.Y(),p2.Z())
                    # print(p3.X(),p3.Y(),p3.Z())
                    # print(p4.X(),p4.Y(),p4.Z())
                    # print("pointend")
                    # print(p5.X(),p5.Y(),p5.Z())
                    # print(p6.X(),p6.Y(),p6.Z())
                    # print(p7.X(),p7.Y(),p7.Z())
                    # print(p8.X(),p8.Y(),p8.Z())
 
                    # print(path_result_2)
                    if i==1:
                        corner_start=point_starts
                    S_corner=grinding_corner_area(point_starts,corner_start)
                    p1=corner_start[0]
                    p2=corner_start[1]
                    p3=corner_start[2]
                    p4=corner_start[3]

                    # print("cornerstart")
                    # print(p1.X(),p1.Y(),p1.Z())
                    # print(p2.X(),p2.Y(),p2.Z())
                    # print(p3.X(),p3.Y(),p3.Z())
                    # print(p4.X(),p4.Y(),p4.Z())

                    
                    corner_start=point_ends

                # S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
                    
                if i==1:
                    path_result=S
                    

                    path_result_2=BRepPrimAPI_MakeBox(0.0001, 0.0001, 0.0001).Shape()
                    # print(p1.X(),p1.Y(),p1.Z())
                    # print(p2.X(),p2.Y(),p2.Z())
                    # print(p3.X(),p3.Y(),p3.Z())
                    # print(p4.X(),p4.Y(),p4.Z())
                    # print(p5.X(),p5.Y(),p5.Z())
                    # print(p6.X(),p6.Y(),p6.Z())
                    # print(p7.X(),p7.Y(),p7.Z())
                    # print(p8.X(),p8.Y(),p8.Z())                      
                    # print(a)
                    # print(b)
                    # print(i)

                        # all_result = all_result.Shape()
                else:
                    path_result = BRepAlgoAPI_Fuse(path_result,S)
                    path_result = path_result.Shape()
                    # print(path_result_2)
                    # print(S_corner)

            
                    path_result_2 = BRepAlgoAPI_Fuse(path_result_2,S_corner)
                    path_result_2 = path_result_2.Shape()
                    # print(path_result_2)
                    # print(S_corner)
                    # print(a)
                    # print(b)
            
        print(b)
        # if b==1:
        if surface_result==None:
            surface_result=path_result
            surface_result_2=path_result_2
            # all_result = all_result.Shape()
        else:
            surface_result = BRepAlgoAPI_Fuse(path_result,surface_result)
            surface_result = surface_result.Shape()
            surface_result_2 = BRepAlgoAPI_Fuse(path_result_2,surface_result_2)
            surface_result_2 = surface_result_2.Shape()
    if a==1:
        all_grind_area=surface_result
        all_grind_area_2=surface_result_2
        # all_result = all_result.Shape()
    else:
        all_grind_area = BRepAlgoAPI_Fuse(all_grind_area,surface_result)
        all_grind_area = all_grind_area.Shape()
        all_grind_area_2 = BRepAlgoAPI_Fuse(all_grind_area_2,surface_result_2)
        all_grind_area_2 = all_grind_area_2.Shape()
all_grind_area = BRepAlgoAPI_Fuse(all_grind_area,all_grind_area_2)
all_grind_area = all_grind_area.Shape()

display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(all_grind_area, update=True)[0]
box.SetTransparency(0.8)
start_display()
    
    # if i < len(same_path_points) - 1:
    #     next_point = [float(item) for item in same_path_points[i + 1]]  # 获取后一个 point 的信息
    #     print("Next Point:", next_point)

[('1', '1', '51.86', '-0.00', '-22.41', '0.81', '-0.00', '-0.59'), ('1', '1', '54.81', '-11.30', '-15.74', '0.87', '-0.18', '-0.45'), ('1', '1', '56.88', '-15.10', '-8.35', '0.93', '-0.25', '-0.26'), ('1', '1', '58.31', '-14.08', '-0.78', '0.97', '-0.23', '-0.03'), ('1', '1', '58.42', '-13.68', '-0.00', '0.97', '-0.23', '0.00')]
ok
ok
ok
1
[('1', '2', '38.80', '-0.00', '-35.58', '0.61', '-0.00', '-0.79'), ('1', '2', '41.33', '-12.69', '-31.90', '0.64', '-0.20', '-0.74'), ('1', '2', '44.00', '-22.45', '-25.54', '0.68', '-0.35', '-0.64'), ('1', '2', '46.28', '-28.53', '-18.63', '0.73', '-0.45', '-0.51'), ('1', '2', '48.29', '-31.96', '-11.29', '0.78', '-0.52', '-0.34'), ('1', '2', '49.95', '-32.79', '-3.84', '0.83', '-0.54', '-0.13'), ('1', '2', '50.66', '-32.14', '-0.00', '0.84', '-0.54', '0.00')]
ok
ok
ok
ok
ok
2
[('1', '3', '26.95', '-0.00', '-43.01', '0.45', '-0.00', '-0.89'), ('1', '3', '28.49', '-8.85', '-41.48', '0.47', '-0.14', '-0.87'), ('1', '3', '30.99', '-20.11', '-36.96', '0

In [39]:
display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(all_grind_area, update=True)[0]
box.SetTransparency(0.8)
start_display()

In [10]:
# def grind(model, S):
#     if model.IsNull():
#         print("错误：模型形状为空。")
#         return None
    
#     if S.IsNull():
#         print("错误：剪切形状为空。")
#         return None

#     cut_algo = BRepAlgoAPI_Cut(model, S)
#     cut_algo.Build()

#     if not cut_algo.IsDone():
#         print("错误：剪切操作失败。")
#         return None

#     cut_result = cut_algo.Shape()
    
#     if cut_result.IsNull():
#         print("警告：剪切结果为空。")
#         return None
    
#     return cut_result

#### 對每一殼層進行掃除，並儲存掃除結果

In [40]:
from OCC.Extend.DataExchange import write_iges_file
import os
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Extend.DataExchange import read_iges_file
from OCC.Core.Quantity import Quantity_NOC_RED, Quantity_NOC_BLUE, Quantity_NOC_GREEN
from OCC.Core.gp import gp_Pnt
mag=0.9875
m_center= gp_Pnt(0,0,61.5)
#讀取模型
iges_file =  r"C:\alan\ITRI\ITRI-test\model\complex_3.IGS"
model = read_iges_file(iges_file)
model=model[0]
Model=zoom(model,m_center,mag)
print("Model=",Model)
print("all_grind_area=",all_grind_area)
cut_result=grind(Model,all_grind_area)
print(cut_result)

<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9875
Model= <class 'TopoDS_Compound'>
all_grind_area= <class 'TopoDS_Compound'>
Model type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
Cut shape type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
<class 'TopoDS_Compound'>


In [27]:
display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(all_grind_area, update=True)[0]
box.SetTransparency(0.8)
start_display()

In [41]:
from OCC.Extend.DataExchange import write_iges_file
import os
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Extend.DataExchange import read_iges_file
from OCC.Core.Quantity import Quantity_NOC_RED, Quantity_NOC_BLUE, Quantity_NOC_GREEN


#讀取模型
iges_file =  r"C:\alan\ITRI\ITRI-test\model\complex_3.IGS"
model = read_iges_file(iges_file)
model=model[0]#如果type(model)=<class 'list'>就要加這行，正常應該是type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>

display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(model, update=True)[0]
start_display()

layer_n=6
# point_start=np.array((-40,50,0))
# point_end=np.array((40,50,0))
# normal_vector=np.array((0,1,0))
# S=grinding_area(point_start,point_end,normal_vector)#生成研磨範圍
m_center= gp_Pnt(0,0,61.5)#質心位置
#simple_1:m_center=(0,0,0)
#simple_2:m_center=(0,-2.2,-2.2)
#simple_3:m_center=(-0.36,-0.76,-0.36)
#complex_2:m_center= gp_Pnt(0, 0, 50)
#complex_3:m_center= gp_Pnt(0, 0, 61.5)
cut_results=[]
surface_area_origin=[]
surface_area_grind=[]
colors = [Quantity_NOC_RED, Quantity_NOC_BLUE, Quantity_NOC_GREEN]  # 定义颜色列表
i=0
while i<layer_n:
    mag=1-i*0.0025
    # mag=1-i*0
    print(model,m_center,mag)
    Model=zoom(model,m_center,mag)
    print("Model=",Model)
    
    cut_result=grind(Model,all_grind_area)
    cut_results.append(cut_result)

    print("cut_result=",cut_result)
    # surface_origin = calculate_surface_area(Model)
    # surface_area_origin.append(surface_origin)
    # print(f"Model_{i}_origin_area:", surface_origin) 
    # surface_cut_result=calculate_surface_area(cut_result)
    # surface_area_grind.append(surface_cut_result)
    # print(f"Model_{i}_final_area:", surface_cut_result) #先不計算表面積


    # display, start_display, add_menu, add_function_to_menu = init_display()
    # box=display.DisplayShape(cut_result, update=True)[0]
    # box.SetTransparency(0.1)
    # start_display()


    #不需要合併模型
    # if i==0:
    #     all_result=cut_result
    #     # all_result = all_result.Shape()
    # else:
    #     print("all_result:",all_result,"cut:",cut_result)
    #     all_result = BRepAlgoAPI_Fuse(all_result,cut_result)
    #     all_result = all_result.Shape()

    # iges_file_path=os.path.join(r"C:\alan\ITRI\ITRI-test\cut_result",f'layer_{i}.igs' )
    # write_iges_file(cut_result, iges_file_path)


    # if surface_origin== surface_cut_result:
    #     break

    i=i+1
# print(surface_area_origin)
# iges_file_path=os.path.join(r"C:\alan\ITRI\ITRI-test\cut_result\all.igs" )
# write_iges_file(all_result, iges_file_path)



<class 'TopoDS_Compound'> <class 'gp_Pnt'> 1.0
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 1.0
Model= <class 'TopoDS_Compound'>
Model type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
Cut shape type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
cut_result= <class 'TopoDS_Compound'>
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9975
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9975
Model= <class 'TopoDS_Compound'>
Model type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
Cut shape type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
cut_result= <class 'TopoDS_Compound'>
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.995
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.995
Model= <class 'TopoDS_Compound'>
Model type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
Cut shape type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
cut_result= <class 'TopoDS_Compound'>
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9925
<class 'TopoDS_Compound'> <class 'gp_Pnt'> 0.9925
Model= <class 'TopoDS_Compound'>
Model type: <class 'OCC.Co

In [18]:
cut_result=grind(Model,all_grind_area)
display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(cut_result, update=True)[0]
box.SetTransparency(0.8)
start_display()

Model type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>
Cut shape type: <class 'OCC.Core.TopoDS.TopoDS_Compound'>


ValueError: invalid null reference in method 'new_AIS_Shape', argument 1 of type 'TopoDS_Shape const &'

#### 檢視各層研磨結果

In [ ]:
from OCC.Core.Quantity import Quantity_Color
from OCC.Extend.DataExchange import read_iges_file
from OCC.Display.SimpleGui import init_display
#讀取模型
iges_file = r"C:\alan\ITRI\ITRI-test\cut_result\layer_0.igs"
cut_result = read_iges_file(iges_file)
print(cut_result)

display, start_display, add_menu, add_function_to_menu = init_display()
box=display.DisplayShape(cut_result, update=True)[0]
box.SetTransparency(0.8)
start_display()


[<class 'TopoDS_Compound'>]


In [ ]:
#一次顯示所有殼層(彩色)
import os
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file
from OCC.Core.Quantity import Quantity_Color
from OCC.Display.OCCViewer import rgb_color

# 初始化显示窗口
display, start_display, add_menu, add_function_to_menu = init_display()

# 读取模型
iges_file_0 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_0.igs"
cut_result_0 = read_iges_file(iges_file_0)

iges_file_1 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_1.igs"
cut_result_1 = read_iges_file(iges_file_1)

iges_file_2 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_2.igs"
cut_result_2 = read_iges_file(iges_file_2)

iges_file_3 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_3.igs"
cut_result_3 = read_iges_file(iges_file_0)

iges_file_4 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_4.igs"
cut_result_4 = read_iges_file(iges_file_4)

iges_file_5 = r"C:\alan\ITRI\ITRI-test\cut_result\layer_5.igs"
cut_result_5 = read_iges_file(iges_file_5)

# 使用 rgb_color 函数设置颜色并显示模型
display.DisplayShape(cut_result_0, update=True, color=rgb_color(0, 1, 0))  
display.DisplayShape(cut_result_1, update=True, color=rgb_color(0, 0, 1))  
display.DisplayShape(cut_result_2, update=True, color=rgb_color(0, 0, 0.6)) 
display.DisplayShape(cut_result_3, update=True, color=rgb_color(0, 0, 0.4))  
display.DisplayShape(cut_result_4, update=True, color=rgb_color(0, 0, 0.2))
display.DisplayShape(cut_result_5, update=True, color=rgb_color(1, 0, 0))    

# 开始显示窗口
start_display()

In [ ]:
# #一次顯示單一殼層
# from OCC.Core.Quantity import Quantity_Color
# from OCC.Extend.DataExchange import read_iges_file
# #讀取模型
# iges_file = r"C:\alan\ITRI\ITRI-test\cut_result\layer_5.igs"
# cut_result = read_iges_file(iges_file)
# display, start_display, add_menu, add_function_to_menu = init_display()
# box=display.DisplayShape(cut_result, update=True)[0]
# box.SetTransparency(0.8)
# start_display()

#### 計算研磨成功率

In [ ]:
success_rate=((surface_area_origin[0]-surface_area_grind[0])-(surface_area_origin[4]-surface_area_grind[4]))/surface_area_origin[0]
print(success_rate)

0.9042213364842218
